# AWS KMS (Key Management Service) Tutorial
---

- AWS KMS Documentation: https://docs.aws.amazon.com/ko_kr/kms/latest/developerguide/overview.html
- boto3 Documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

In [ ]:
# -*- coding: utf-8 -*-
import boto3
import pprint
import base64

kms_client = boto3.client('kms')

## CMK (Customer Master Key) 조회
---

In [ ]:
resp = kms_client.list_keys()

for cmk in resp["Keys"]:
	pprint.pprint(cmk)

## Data Key 생성
---

In [ ]:
# KeyId에는 KeyArn이나 KeyId가 들어간다.
# 제일 첫번째 CMK를 사용하자
resp = kms_client.generate_data_key(
    KeyId=resp["Keys"][0]["KeyId"],
    KeySpec='AES_256'
)

# 암호화된 Data key
data_key_encrypted = resp["CiphertextBlob"]

# 복호화된 Data Key
data_key_plaintext = resp["Plaintext"]

# Data Key는 바이너리 형태이므로 읽을 수 있게 Base64로 인코딩
data_key_base64 = base64.b64encode(data_key_plaintext)

pprint.pprint(resp)
print("\n\nPlain Data Key(Base64):", data_key_base64)

## 암호화된 Data Key의 복호화
---

In [ ]:
resp = kms_client.decrypt(CiphertextBlob=data_key_encrypted)

# 복호화된 Data Key
data_key_plaintext = resp["Plaintext"]

# Data Key는 바이너리 형태이므로 읽을 수 있게 Base64로 인코딩
data_key_base64 = base64.b64encode(data_key_plaintext)

print("\n\nPlain Data Key(Base64):", data_key_base64)

## 파일 암호화
---

- cryptography Documentation: https://cryptography.io/en/latest/

In [ ]:
import cryptography.fernet

NUM_BYTES_FOR_LEN = 4
PLAIN_SAMPLE_FILE_NAME = "sample_plain.txt"  # 읽을 샘플 파일명
ENCRYPTED_FILE_NAME = "sample_encrypted.txt"  # 암호화하여 저장할 파일명

file_contents_encrypted = b""

with open(PLAIN_SAMPLE_FILE_NAME, "rb") as f:
    file_contents = f.read()

    # Data Key를 Base64로 인코딩한 값으로 샘플 파일을 암호화
    fernet = cryptography.fernet.Fernet(data_key_base64)
    file_contents_encrypted = fernet.encrypt(file_contents)

with open(ENCRYPTED_FILE_NAME, "wb") as f:
    # CMK로 암호화된 Data Key의 길이
    f.write(len(data_key_encrypted).to_bytes(NUM_BYTES_FOR_LEN, byteorder="big"))
    
    # CMK로 암호화된 Data Key
    f.write(data_key_encrypted)
    
    # Data Key를 Base64로 인코딩한 값으로 암호화된 값
    f.write(file_contents_encrypted)

print("Done")

## 파일 복호화
---

In [ ]:
NUM_BYTES_FOR_LEN = 4
ENCRYPTED_FILE_NAME = "sample_encrypted.txt"  # 저장한 암호화된 샘플 파일명

data_key_encrypted = b""
file_contents_encrypted = b""

with open(ENCRYPTED_FILE_NAME, "rb") as f:
    file_contents = f.read()

    # CMK로 암호화된 Data Key의 길이
    data_key_encrypted_len = int.from_bytes(file_contents[:NUM_BYTES_FOR_LEN], byteorder="big")
    
    # CMK로 암호화된 Data Key
    data_key_encrypted = file_contents[NUM_BYTES_FOR_LEN:NUM_BYTES_FOR_LEN+data_key_encrypted_len]
    
    # Data Key를 Base64로 인코딩한 값으로 암호화된 값
    file_contents_encrypted = file_contents[NUM_BYTES_FOR_LEN+data_key_encrypted_len:]

# 암호화된 Data Key의 복호화
resp = kms_client.decrypt(CiphertextBlob=data_key_encrypted)
data_key_plaintext = resp["Plaintext"]
data_key_base64 = base64.b64encode(data_key_plaintext)

# 복호화된 Data Key를 Base64로 인코딩한 값으로 샘플 파일을 복호화
fernet = cryptography.fernet.Fernet(data_key_base64)
file_contents_decrypted = fernet.decrypt(file_contents_encrypted)

print("Result:", file_contents_decrypted.decode())